<a href="https://colab.research.google.com/github/haromero/Juegos-Inteligentes/blob/main/Reversi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reglas para la creacion del juego

- El tablero tiene que ser 8x8.

- Las fichas iniciales tienes que estar en el centro del tablero, de forma diagonal 2 negras y 2 blancas.

- La ficha negra siempre empieza.

- Para convertir una ficha en la tuya debe haber una tuya en una flanco opuesto cubriendo a la del rival en los dos flancos para transformarlas en la tuya (tanto como en diagonal, vertical y horizontal).

- Si un jugador tiene mas fichas y no hay mas movimientos o el tablero esta lleno ese jugador gana.

Dos jugadores:

Jugador 1: Negro (○)

Jugador 2: Blanco (●)

# Librerias a importar

In [ ]:
import time
from IPython.display import clear_output
import copy
import random
import sys
import os
import numpy as np
import pandas as pd
import os
try:
    from IPython.display import clear_output
except ImportError:
    clear_output = None  # Para entornos sin IPython

# Clase ReversiGame

In [ ]:
class ReversiGame:
    def __init__(self, size=8):
        """Inicializa un juego de Reversi con un tablero de tamaño dado (mínimo 4x4, par)."""
        if size < 4 or size % 2 != 0:
            raise ValueError("El tamaño del tablero debe ser par y al menos 4x4.")
        self.size = size
        self.board = [['-' for _ in range(self.size)] for _ in range(self.size)]
        self.initialize_board()
        self.current_player = '○'  # Negro empieza
        self.opponent = '●'

    def initialize_board(self):
        """Coloca las 4 fichas iniciales en el centro del tablero en disposición diagonal."""
        mid = self.size // 2
        self.board[mid-1][mid-1] = '●'  # Blanca
        self.board[mid][mid] = '●'      # Blanca
        self.board[mid-1][mid] = '○'    # Negra
        self.board[mid][mid-1] = '○'    # Negra

    def display_board(self, show_valid_moves=False):
        """Muestra el tablero con un diseño elegante en consola. Opcionalmente muestra movimientos válidos con números."""
        try:
            # clear_output(wait=True)
            pass
        except:
            os.system('cls' if os.name == 'nt' else 'clear')

        # Preparar tablero para mostrar
        display_board = [row[:] for row in self.board]

        # Reemplazar '-' por ' '
        for r in range(self.size):
            for c in range(self.size):
                if display_board[r][c] == '-':
                    display_board[r][c] = ' '

        if show_valid_moves:
            valid_moves = self.get_valid_moves()
            for i, (r, c) in enumerate(valid_moves, 1):
                display_board[r][c] = str(i)

        # Encabezado de columnas
        print("     " + "   ".join(str(i) for i in range(self.size)))
        print("   +" + "---+" * self.size)

        # Filas del tablero
        for idx, row in enumerate(display_board):
            print(f" {idx} | " + " | ".join(row) + " |")
            print("   +" + "---+" * self.size)

        print(f"\nTurno: {'Negro' if self.current_player == '○' else 'Blanco'}")
        if show_valid_moves:
            print("(Los números indican movimientos válidos)")

    def get_valid_moves(self):
        """Devuelve una lista ordenada de movimientos válidos (fila, columna)."""
        return sorted([(r, c) for r in range(self.size) for c in range(self.size) if self.is_valid_move(r, c)],
                      key=lambda x: (x[0], x[1]))

    def switch_player(self):
        """Cambia el turno entre el jugador actual y el oponente."""
        self.current_player, self.opponent = self.opponent, self.current_player

    def is_valid_move(self, row, col):
        """Verifica si un movimiento en (row, col) es legal según las reglas de Reversi."""
        if not (0 <= row < self.size and 0 <= col < self.size):
            return False
        if self.board[row][col] != '-':
            return False
        directions = [(-1, 0), (1, 0), (0, -1), (0, 1),
                      (-1, -1), (-1, 1), (1, -1), (1, 1)]
        for dr, dc in directions:
            r, c = row + dr, col + dc
            has_opponent_between = False
            while 0 <= r < self.size and 0 <= c < self.size:
                if self.board[r][c] == self.opponent:
                    has_opponent_between = True
                elif self.board[r][c] == self.current_player:
                    if has_opponent_between:
                        return True
                    break
                else:
                    break
                r += dr
                c += dc
        return False

    def make_move(self, row, col):
        """Coloca una ficha en (row, col) y voltea las fichas del oponente si el movimiento es válido."""
        if not (0 <= row < self.size and 0 <= col < self.size):
            return False
        if not self.is_valid_move(row, col):
            return False
        self.board[row][col] = self.current_player
        directions = [(-1, 0), (1, 0), (0, -1), (0, 1),
                      (-1, -1), (-1, 1), (1, -1), (1, 1)]
        for dr, dc in directions:
            self.flip_direction(row, col, dr, dc)
        self.switch_player()
        return True

    def flip_direction(self, row, col, dr, dc):
        """Voltea las fichas del oponente en una dirección específica si es posible."""
        r, c = row + dr, col + dc
        pieces_to_flip = []
        while 0 <= r < self.size and 0 <= c < self.size:
            if self.board[r][c] == self.opponent:
                pieces_to_flip.append((r, c))
            elif self.board[r][c] == self.current_player:
                for pr, pc in pieces_to_flip:
                    self.board[pr][pc] = self.current_player
                return len(pieces_to_flip)
            else:
                return 0
            r += dr
            c += dc
        return 0

    def has_valid_moves(self):
        """Verifica si el jugador actual tiene movimientos válidos."""
        return any(self.is_valid_move(r, c) for r in range(self.size) for c in range(self.size))

In [ ]:
game = ReversiGame()

while True:
    game.display_board(show_valid_moves=True)
    if not game.has_valid_moves():
        print("No hay movimientos válidos. Pasando turno.")
        game.switch_player()
        continue

    valid_moves = game.get_valid_moves()
    try:
        move_input = input("Ingresa el número de la jugada (o 'q' para salir): ")

        if move_input.lower() in ['q', 'quit', 'exit']:
            print("Juego terminado por el usuario.")
            break

        move_num = int(move_input)
        if 1 <= move_num <= len(valid_moves):
            row, col = valid_moves[move_num - 1]
            if game.make_move(row, col):
                # Continuar el juego
                pass
            else:
                print("Movimiento inválido. Intenta de nuevo.")
        else:
            print("Número inválido. Intenta de nuevo.")
    except ValueError:
        print("Entrada inválida. Debe ser un número.")

     0   1   2   3   4   5   6   7
   +---+---+---+---+---+---+---+---+
 0 |   |   |   |   |   |   |   |   |
   +---+---+---+---+---+---+---+---+
 1 |   |   |   |   |   |   |   |   |
   +---+---+---+---+---+---+---+---+
 2 |   |   |   | 1 |   |   |   |   |
   +---+---+---+---+---+---+---+---+
 3 |   |   | 2 | ● | ○ |   |   |   |
   +---+---+---+---+---+---+---+---+
 4 |   |   |   | ○ | ● | 3 |   |   |
   +---+---+---+---+---+---+---+---+
 5 |   |   |   |   | 4 |   |   |   |
   +---+---+---+---+---+---+---+---+
 6 |   |   |   |   |   |   |   |   |
   +---+---+---+---+---+---+---+---+
 7 |   |   |   |   |   |   |   |   |
   +---+---+---+---+---+---+---+---+

Turno: Negro
(Los números indican movimientos válidos)
Ingresa el número de la jugada (o 'q' para salir): q
Juego terminado por el usuario.


# Ejemplo de Ogando

In [ ]:
import numpy as np
import time

class MinimaxSolver():

  def __init__(self, player_name):

    self.player_name = player_name
    self.time_start = None
    self.max_time = None

  def __maximize(self, state, alpha, beta, depth):

    if time.time() - self.time_start >= self.max_time:
      raise StopIteration("Out of time!")

    if state.is_terminal():
      return None, state.get_winners_points()[self.player_name]

    if depth <= 0:
      return None, state.heuristic(self.player_name)

    max_child, max_utility = None, -np.inf

    for option, child in state.children():

      if child.curr_player.name == self.player_name:
        _, utility = self.__maximize(child, alpha, beta, depth-1)
      else:
        _, utility = self.__minimize(child, alpha, beta, depth-1)

      if utility > max_utility:
        max_child, max_utility = option, utility

      if max_utility >= beta:
        break

      alpha = max(alpha, max_utility)

    return max_child, max_utility

  def __minimize(self, state, alpha, beta, depth):

    if time.time() - self.time_start >= self.max_time:
      raise StopIteration("Out of time!")

    if state.is_terminal():
      return None, state.get_winners_points()[self.player_name]

    if depth <= 0:
      return None, state.heuristic(self.player_name)

    min_child, min_utility = None, np.inf

    for option, child in state.children():

      if child.curr_player.name == self.player_name:
        _, utility = self.__maximize(child, alpha, beta, depth-1)
      else:
        _, utility = self.__minimize(child, alpha, beta, depth-1)

      if utility < min_utility:
        min_child, min_utility = option, utility

      if min_utility <= alpha:
        break

      beta = min(beta, min_utility)

    return min_child, min_utility

  def solve(self, state, max_time):

    self.time_start = time.time()
    self.max_time = max_time
    for depth in range(2, 10000):
      try:
        best_option, _ = self.__maximize(state, -np.inf, np.inf, depth)
      except StopIteration:
        break

    return best_option

In [ ]:
game = PresidentGameSession(4, minimax_time=2)
game.start_loop()

# **Minimax con poda Alfa-Beta + IDS**

In [ ]:
# --- Minimax con poda Alfa-Beta + IDS ---
class MinimaxAgent:
    """
    Implementa el algoritmo Minimax con:
    - Poda Alfa-Beta para reducir nodos.
    - IDS (Iterative Deepening Search) con límite de tiempo y profundidad.
    - Heurísticas configurables mediante pesos.
    - Estadísticas: nodos expandidos, profundidad alcanzada y tiempo por jugada.
    """

    def __init__(self, symbol, max_time=2.0, max_depth=6, name="Minimax", weights=None):
        self.symbol = symbol                 # '○' (negro) o '●' (blanco)
        self.max_time = float(max_time)      # tiempo máximo por jugada (segundos)
        self.max_depth = int(max_depth)      # límite máximo de profundidad
        self.name = name                     # nombre (para reportes y benchmark)

        # --- Estadísticas por partida ---
        self.start_time = None
        self.nodes_expanded = 0
        self.max_depth_reached = 0
        self.total_time = 0.0  # acumulado en milisegundos

        # --- Heurísticas (por defecto: diferencia de fichas + movilidad) ---
        self.weights = weights if weights else {
            "disk_diff": 1.0,
            "mobility": 0.5
        }

    # ==========================
    # Heurística modular
    # ==========================
    def heuristic(self, game):
        """
        Evalúa el tablero con una suma ponderada de factores activados en self.weights.
        """
        score = 0.0
        me = self.symbol
        opp = '○' if me == '●' else '●'
        n = game.size

        # 1) Diferencia de fichas
        if "disk_diff" in self.weights:
            my_count  = sum(row.count(me)  for row in game.board)
            opp_count = sum(row.count(opp) for row in game.board)
            score += self.weights["disk_diff"] * (my_count - opp_count)

        # 2) Movilidad (jugadas posibles)
        if "mobility" in self.weights:
            my_moves = len(game.get_valid_moves())
            game.switch_player()
            opp_moves = len(game.get_valid_moves())
            game.switch_player()
            score += self.weights["mobility"] * (my_moves - opp_moves)

        # 3) Esquinas (opcional, si activas en weights)
        if "corners" in self.weights:
            corners = [(0,0),(0,n-1),(n-1,0),(n-1,n-1)]
            my_c = sum(1 for r,c in corners if game.board[r][c] == me)
            op_c = sum(1 for r,c in corners if game.board[r][c] == opp)
            score += self.weights["corners"] * (my_c - op_c)

        # 4) Cercanía a esquinas (opcional)
        if "corner_closeness" in self.weights:
            near = [(0,1),(1,0),(1,1),
                    (0,n-2),(1,n-2),(1,n-1),
                    (n-2,0),(n-2,1),(n-1,1),
                    (n-2,n-1),(n-2,n-2),(n-1,n-2)]
            my_d = sum(1 for r,c in near if game.board[r][c] == me)
            op_d = sum(1 for r,c in near if game.board[r][c] == opp)
            score += self.weights["corner_closeness"] * (my_d - op_d)

        # 5) Bordes (opcional)
        if "edges" in self.weights:
            my_e = op_e = 0
            for i in range(n):
                if game.board[i][0]   == me:  my_e += 1
                elif game.board[i][0] == opp: op_e += 1
                if game.board[i][n-1]   == me:  my_e += 1
                elif game.board[i][n-1] == opp: op_e += 1
                if game.board[0][i]   == me:  my_e += 1
                elif game.board[0][i] == opp: op_e += 1
                if game.board[n-1][i]   == me:  my_e += 1
                elif game.board[n-1][i] == opp: op_e += 1
            score += self.weights["edges"] * (my_e - op_e)

        return score

    # ==========================
    # Elección de jugada con IDS
    # ==========================
    def choose(self, game):
        """
        Iterative Deepening: profundiza de 1 a max_depth o hasta agotar tiempo.
        Devuelve la mejor jugada encontrada.
        """
        self.start_time = time.time()
        self.nodes_expanded = 0
        self.max_depth_reached = 0

        best_move = None
        depth = 1
        try:
            while depth <= self.max_depth:
                move, _ = self._maximize(game, -np.inf, np.inf, depth)
                if move is not None:
                    best_move = move
                self.max_depth_reached = max(self.max_depth_reached, depth)
                depth += 1
        except TimeoutError:
            pass  # se queda con lo mejor hallado

        self.total_time += (time.time() - self.start_time) * 1000.0  # ms
        return best_move

    # ==========================
    # Helpers internos
    # ==========================
    def _time_up(self):
        if (time.time() - self.start_time) >= self.max_time:
            raise TimeoutError

    def _ordered_moves(self, game, maximize=True):
        """
        Ordena jugadas según heurística de la posición hija → mejora la poda.
        """
        moves = game.get_valid_moves()
        scored = []
        for m in moves:
            g2 = copy.deepcopy(game)
            g2.make_move(*m)
            scored.append((self.heuristic(g2), m))
        scored.sort(reverse=maximize, key=lambda x: x[0])
        return [m for _, m in scored]

    # ==========================
    # Minimax recursivo con poda α–β
    # ==========================
    def _maximize(self, game, alpha, beta, depth):
        self._time_up()
        self.nodes_expanded += 1

        if depth == 0 or not game.has_valid_moves():
            return None, self.heuristic(game)

        best_move, best_val = None, -np.inf
        for move in self._ordered_moves(game, maximize=True):
            g2 = copy.deepcopy(game)
            g2.make_move(*move)
            _, val = self._minimize(g2, alpha, beta, depth-1)

            if val > best_val:
                best_val, best_move = val, move
            if best_val >= beta:  # poda β
                break
            alpha = max(alpha, best_val)
        return best_move, best_val

    def _minimize(self, game, alpha, beta, depth):
        self._time_up()
        self.nodes_expanded += 1

        if depth == 0 or not game.has_valid_moves():
            return None, self.heuristic(game)

        best_move, best_val = None, np.inf
        for move in self._ordered_moves(game, maximize=False):
            g2 = copy.deepcopy(game)
            g2.make_move(*move)
            _, val = self._maximize(g2, alpha, beta, depth-1)

            if val < best_val:
                best_val, best_move = val, move
            if best_val <= alpha:  # poda α
                break
            beta = min(beta, best_val)
        return best_move, best_val


# **Rivales Automáticos**

In [ ]:
# --- Rivales automáticos ---

class RandomAgent:
    """
    Juega completamente al azar.
    Sirve como baseline para comparar con Minimax.
    """
    def __init__(self, symbol):
        self.symbol = symbol
        self.name = "Random"

    def choose(self, game):
        moves = game.get_valid_moves()
        return random.choice(moves) if moves else None


class GreedyAgent:
    """
    Juega de forma codiciosa:
    Elige la jugada que le dé MÁS fichas en ese turno.
    """
    def __init__(self, symbol):
        self.symbol = symbol
        self.name = "Greedy"

    def choose(self, game):
        moves = game.get_valid_moves()
        if not moves:
            return None
        best, best_gain = None, -10**9
        for m in moves:
            g2 = copy.deepcopy(game)
            before = sum(row.count(self.symbol) for row in g2.board)
            g2.make_move(*m)
            after = sum(row.count(self.symbol) for row in g2.board)
            gain = after - before
            if gain > best_gain:
                best_gain, best = gain, m
        return best


class WorstAgent:
    """
    Juega lo peor posible:
    Elige la jugada que le dé MENOS fichas en ese turno.
    (Sirve para probar qué tan bien se defiende Minimax).
    """
    def __init__(self, symbol):
        self.symbol = symbol
        self.name = "Worst"

    def choose(self, game):
        moves = game.get_valid_moves()
        if not moves:
            return None
        worst, worst_gain = None, 10**9
        for m in moves:
            g2 = copy.deepcopy(game)
            before = sum(row.count(self.symbol) for row in g2.board)
            g2.make_move(*m)
            after = sum(row.count(self.symbol) for row in g2.board)
            gain = after - before
            if gain < worst_gain:
                worst_gain, worst = gain, m
        return worst

class HumanAgent:
    """
    Agente controlado por el usuario.
    Permite al jugador humano elegir jugadas manualmente.
    """
    def __init__(self, symbol):
        self.symbol = symbol
        self.name = "Humano"
        self.quit = False  # 🔹 Nuevo: indica si el jugador quiere salir

    def choose(self, game: "ReversiGame"):
        valid_moves = game.get_valid_moves()
        if not valid_moves:
            print(" No tienes jugadas válidas, se pasa el turno.")
            return None

        # Mostrar jugadas disponibles
        print("\nMovimientos válidos:")
        for i, (r, c) in enumerate(valid_moves, 1):
            print(f"{i}: ({r}, {c})")

        # Pedir entrada al usuario
        while True:
            try:
                move_input = input("Elige el número de la jugada (o 'q' para salir): ")
                if move_input.lower() in ['q', 'quit', 'exit']:
                    print("Juego terminado por el jugador.")
                    self.quit = True   # marcar que el jugador salió
                    return None
                move_num = int(move_input)
                if 1 <= move_num <= len(valid_moves):
                    return valid_moves[move_num - 1]
                else:
                    print("Número inválido. Intenta otra vez.")
            except ValueError:
                print("Entrada inválida, escribe un número.")


# **Runner de partidas**

In [ ]:
# --- Runner de partidas con estadísticas extendidas ---
def play_game(agent_black, agent_white, verbose=True):
    """
    Ejecuta una partida de Reversi entre dos agentes (Human, IA o mixto).
    - Incluye estadísticas de Minimax: nodos expandidos y profundidad alcanzada.
    """
    game = ReversiGame()
    game.current_player = '○'
    game.opponent = '●'
    start = time.time()

    while True:
        agent = agent_black if game.current_player == '○' else agent_white
        move = agent.choose(game)

        if isinstance(agent, HumanAgent) and getattr(agent, "quit", False):
            print("\n Partida interrumpida por el jugador humano.")
            break

        if move:
            game.make_move(*move)
            if verbose:
                game.display_board()
        else:
            if not game.has_valid_moves():
                game.switch_player()
                if not game.has_valid_moves():
                    break

    # Estadísticas finales
    elapsed_total = (time.time() - start) * 1000.0
    black_points = sum(row.count('○') for row in game.board)
    white_points = sum(row.count('●') for row in game.board)
    winner = "Tie"
    if black_points > white_points:
        winner = "Black"
    elif white_points > black_points:
        winner = "White"

    return {
        "Black": agent_black.name,
        "White": agent_white.name,
        "Winner": winner,
        "Black_points": black_points,
        "White_points": white_points,
        "Game_time_ms": elapsed_total,
        # Estadísticas del Minimax si está presente
        "Black_nodes": getattr(agent_black, "nodes_expanded", None),
        "White_nodes": getattr(agent_white, "nodes_expanded", None),
        "Black_depth": getattr(agent_black, "max_depth_reached", None),
        "White_depth": getattr(agent_white, "max_depth_reached", None),
        "Black_time_ms": getattr(agent_black, "total_time", None),
        "White_time_ms": getattr(agent_white, "total_time", None),
    }


Test para usar

In [ ]:
# IA vs IA
ai1 = MinimaxAgent('○', max_time=2.0, max_depth=4, name="Minimax-Black")
ai2 = HumanAgent('●')
resultado = play_game(ai1, ai2, verbose=True)
print(resultado)



     0   1   2   3   4   5   6   7
   +---+---+---+---+---+---+---+---+
 0 |   |   |   |   |   |   |   |   |
   +---+---+---+---+---+---+---+---+
 1 |   |   |   |   |   |   |   |   |
   +---+---+---+---+---+---+---+---+
 2 |   |   |   | ○ |   |   |   |   |
   +---+---+---+---+---+---+---+---+
 3 |   |   |   | ○ | ○ |   |   |   |
   +---+---+---+---+---+---+---+---+
 4 |   |   |   | ○ | ● |   |   |   |
   +---+---+---+---+---+---+---+---+
 5 |   |   |   |   |   |   |   |   |
   +---+---+---+---+---+---+---+---+
 6 |   |   |   |   |   |   |   |   |
   +---+---+---+---+---+---+---+---+
 7 |   |   |   |   |   |   |   |   |
   +---+---+---+---+---+---+---+---+

Turno: Blanco

Movimientos válidos:
1: (2, 2)
2: (2, 4)
3: (4, 2)
Elige el número de la jugada (o 'q' para salir): 1
     0   1   2   3   4   5   6   7
   +---+---+---+---+---+---+---+---+
 0 |   |   |   |   |   |   |   |   |
   +---+---+---+---+---+---+---+---+
 1 |   |   |   |   |   |   |   |   |
   +---+---+---+---+---+---+---+---

' IA vs Humano\nai = MinimaxAgent(\'○\', max_time=2.0, max_depth=4, name="Minimax")\nhuman = HumanAgent(\'●\')\nresultado = play_game(ai, human, verbose=True)\nprint(resultado)'

# **Experimentos con heurísticas y pesos**

In [ ]:
# --- Benchmark con diferentes configuraciones de heurísticas ---
def run_benchmark_heuristics(rounds=2, max_time=2.0, max_depth=4, verbose=False):
    """
    Ejecuta un benchmark de Minimax con varias configuraciones de heurísticas
    contra Random, Greedy y Worst.
    Devuelve un DataFrame con resultados.
    """
    # primera configuracion de peso
    '''
    weight_sets = [
        {"disk_diff": 1.0},
        {"mobility": 1.0},
        {"corners": 2.0},
        {"corner_closeness": -1.0},
        {"edges": 1.0},
        {"disk_diff": 1.0, "mobility": 0.5},
        {"disk_diff": 1.0, "mobility": 0.5, "corners": 2.0},
        {"disk_diff": 1.0, "mobility": 0.5, "corners": 2.0, "edges": 0.5},
        {"disk_diff": 1.0, "mobility": 0.5, "corners": 2.0,
         "corner_closeness": -0.5, "edges": 0.5},
    ]
    '''

    # segunda configuracion de peso
    weight_sets = [
        {"disk_diff": 1.0},
        {"mobility": 1.5},
        {"corners": 3.0},
        {"corner_closeness": -1.5},
        {"edges": 1.0},
        {"disk_diff": 1.0, "mobility": 1.0},
        {"disk_diff": 1.0, "mobility": 1.0, "corners": 3.0},
        {"disk_diff": 1.0, "mobility": 1.0, "corners": 3.0, "edges": 1.0},
        {"disk_diff": 1.0, "mobility": 1.0, "corners": 3.0,
         "corner_closeness": -1.0, "edges": 1.0 },
    ]

    rivals = [
        RandomAgent("●"),
        GreedyAgent("●"),
        WorstAgent("●"),
        MinimaxAgent('●', max_time=2.0, max_depth=4, name="Minimax-White")
    ]

    results = []
    for i, weights in enumerate(weight_sets, 1):
        for rival in rivals:
            for r in range(rounds):
                ai = MinimaxAgent("○", max_time=max_time, max_depth=max_depth,
                                  name=f"Minimax_w{i}", weights=weights)
                res = play_game(ai, rival, verbose=verbose)
                res["Config"] = i
                res["Weights"] = str(weights)
                res["Rival"] = rival.name
                results.append(res)

    return pd.DataFrame(results)


In [ ]:
# --- Ejecutar benchmark ---

df_benchmark = run_benchmark_heuristics(rounds=2, max_time=2.0, max_depth=4, verbose=False)

#print("\n Resultados detallados:")
#print(df_benchmark)

# --- Guardar CSV detallado ---
#df_benchmark.to_csv("benchmark_detallado.csv", index=False)

# --- Resumen agregado (incluye nodos y profundidad) ---
summary = df_benchmark.groupby(["Config","Weights","Rival"]).agg({
    "Winner": lambda x: (x == "Black").mean(),   # winrate de Minimax
    "Black_points": "mean",
    "White_points": "mean",
    "Black_nodes": "mean",
    "Black_depth": "mean",
    "Black_time_ms": "mean",
    "Game_time_ms": "mean"
}).reset_index()

summary = summary.rename(columns={"Winner":"WinRate(Black)"})

print("\n Resumen agregado:")
print(summary)

# --- Guardar CSV resumen ---
summary.to_csv("benchmark_resumen.csv", index=False)

print("\n Archivos exportados: 'benchmark_detallado.csv' y 'benchmark_resumen.csv'")



 Resumen agregado:
    Config                                            Weights          Rival  \
0        1                                 {'disk_diff': 1.0}         Greedy   
1        1                                 {'disk_diff': 1.0}  Minimax-White   
2        1                                 {'disk_diff': 1.0}         Random   
3        1                                 {'disk_diff': 1.0}          Worst   
4        2                                  {'mobility': 1.5}         Greedy   
5        2                                  {'mobility': 1.5}  Minimax-White   
6        2                                  {'mobility': 1.5}         Random   
7        2                                  {'mobility': 1.5}          Worst   
8        3                                   {'corners': 3.0}         Greedy   
9        3                                   {'corners': 3.0}  Minimax-White   
10       3                                   {'corners': 3.0}         Random   
11       3          